In [37]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from copy import deepcopy

from tools.classical import *
from tools.quantum import *

#### Qutip Exact Hamiltonian

In [28]:
num_qubits = 5

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
coeffs = [1, 1, 1, 1]
hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeffs, num_qubits=num_qubits, symbreak_term=[Z])
# print(hamiltonian_qt)
#* HAMILTONIANS
exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

#* SPECTRUMS
# print(f'Shifted spectrum {np.linalg.eigvalsh(shifted_hamiltonian_qt)}')
print(f'Rescaling factor {rescaling_factor}')
print(f'Shift {shift}')
print(f'Original spectrum {exact_spec}')
print(f'Normalized spectrum {np.linalg.eigvalsh(rescaled_hamiltonian_qt)}')

Rescaling factor 16.41481
Shift 0.5126352361069059
Original spectrum [-8.41480675e+00 -8.01149686e+00 -7.45710970e+00 -6.56278848e+00
 -4.82842712e+00 -3.76446278e+00 -3.46410162e+00 -3.10903996e+00
 -2.56016642e+00 -2.48908833e+00 -1.72700584e+00 -1.54375254e+00
 -1.12310563e+00 -3.68457240e-01 -1.47629054e-15 -3.06637963e-16
  8.28427125e-01  9.50705136e-01  1.02497194e+00  1.68545843e+00
  1.85582806e+00  2.00000000e+00  2.57314158e+00  3.08156908e+00
  3.46410162e+00  3.87693371e+00  4.00000000e+00  4.00000000e+00
  4.84875524e+00  6.11081172e+00  7.12310563e+00  8.00000000e+00]
Normalized spectrum [1.97901509e-07 2.45700765e-02 5.83436725e-02 1.12826254e-01
 2.18484580e-01 2.83301922e-01 3.01600103e-01 3.23230670e-01
 3.56668373e-01 3.60998493e-01 4.07425012e-01 4.18588912e-01
 4.44214973e-01 4.90188602e-01 5.12635236e-01 5.12635236e-01
 5.63103510e-01 5.70552759e-01 5.75077137e-01 6.15314367e-01
 6.25693387e-01 6.34476427e-01 6.69392553e-01 7.00366259e-01
 7.23670369e-01 7.488203

### Qutip unitary evolution construction and comparison: Exact vs Trotterized

$n = 3, k = 4$
Hamiltonian is rescaled already by its coupling strength, not by the rescaled time anymore (bit easier to have an overview)
$$H = 0.125\times Heisenberg_3 + Z_{bulk}$$

In [38]:
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 100
step_size = total_time / num_trotter_steps
rescaled_coeffs = coeffs / rescaling_factor

# rescaled_total_time = total_time / rescaling_factor
# rescaled_step_size = rescaled_total_time / num_trotter_steps

#* EXACT UNITARIES
exact_U_qt = expm(1j * rescaled_hamiltonian_qt * total_time)

#* QUTIP TROTTER
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, coeffs=rescaled_coeffs, shift=(total_time*shift))
# UUdag_minus_id = exact_U_qt * trott_U_qt.dag() - qt.qeye(exact_U_qt.shape[0])

#* QUTIP DISTANCES
dist_between_unitaries = np.linalg.norm(exact_U_qt - trott_U_qt)
print(f'Distance between Qutip Trotter vs Exact {dist_between_unitaries}')

# with np.printoptions(precision=3, suppress=True):
#     print(exact_U_qt)
#     print(trott_U_qt)
#     print(UUdag_minus_id)
    
# logU_shifted = logm(shifted_trott_U_qt) / (1j * T)
# logU = logm(trott_U_qt) / (1j * T)
# dist_shifted = np.linalg.norm(logU_shifted - shifted_exact_U_qt)
# dist = np.linalg.norm(logU - exact_U_qt)
# print(dist_shifted)
# print(dist)

Distance between Qutip Trotter vs Exact 0.04089813777483609


### Observable expectation comparison: Exact vs Trotterized, finding a good number of Trotter steps

In [39]:
#* Initial state
# initial_state = np.ones(2**num_qubits) / np.sqrt(2**num_qubits)  # Equal superposition
np.random.seed(666)
randstate_better = np.zeros(2**num_qubits)
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
print(randstate_better)
randstate_better /= np.linalg.norm(randstate_better)
rand_ini_state = randstate_better
rand_ini_state_qt = rand_ini_state

initial_energy = rand_ini_state.conj().T @ rescaled_hamiltonian_qt.data @ rand_ini_state
print(f'Initial energy {initial_energy}')

# Pretty garbage:
# rand_state0 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state0 /= np.linalg.norm(rand_state0)
# rand_state1 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state1 /= np.linalg.norm(rand_state1)
# print(qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(rand_state1)))

#* Evolved states
exact_evolved_state = qt.Qobj(exact_U_qt) * qt.Qobj(rand_ini_state_qt)
trott_evolved_state = qt.Qobj(trott_U_qt) * qt.Qobj(rand_ini_state_qt)

#* Fidelities
exact_evolved_fidelity = qt.fidelity(exact_evolved_state, trott_evolved_state)
# print('Shifted:')  #* Global phase won't show up in fidelity
# fid_of_states_shifted = qt.fidelity(exact_evolved_state_shifted, trott_evolved_state_shifted)
print(f'Exact-Evolved fidelity {exact_evolved_fidelity}')


[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0.]
Initial energy (0.680166873695157+0j)
Exact-Evolved fidelity 0.9999892595673487


#### Circuit

In [31]:
shift_1q = lambda shift: np.diag(np.ones(2) * np.exp(1j * shift))

In [40]:
trotter_step = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeffs, symbreak=True)
ham_time_evol = ham_evol(num_qubits, trotter_step=trotter_step, num_trotter_steps=num_trotter_steps, time=total_time)

circ = QuantumCircuit(num_qubits, name="H")

for q in range(num_qubits):
    phase_gate = Operator(shift_1q(shift*total_time))
    circ.append(phase_gate, [q])
    
circ.compose(ham_time_evol, range(num_qubits), inplace=True)

print(f'#Qubits {num_qubits}')
print(f'#Trotter steps {num_trotter_steps}')
print(f'#Gates {len([(op, _, _) for (op, _, _) in circ.data])}')
# print(circ)


#Qubits 5
#Trotter steps 100
#Gates 1805


### Comparing Circuit unitary operator to Exact

In [45]:
#* Circuit operators
trotterized_evolution_unitary = Operator(circ).data

#* Distance to exact unitary
dist_circ_unitary_vs_exact = np.linalg.norm(trotterized_evolution_unitary - exact_U_qt)

#* Distance to qutip trotter
dist_circ_vs_qutip_trotter = np.linalg.norm(trotterized_evolution_unitary - trott_U_qt)  # Correct without shift
circ_qutip_dag = np.abs((qt.Qobj(trotterized_evolution_unitary) * qt.Qobj(trott_U_qt).dag()).full()) - qt.qeye(trotterized_evolution_unitary.shape[0])

print(f'Distance between circ unitary and exact {dist_circ_unitary_vs_exact}')
print(f'Distance between circ unitary and qutip trotter {dist_circ_vs_qutip_trotter}')
print(circ_qutip_dag)
print(np.linalg.norm(circ_qutip_dag))

Distance between circ unitary and exact 1.789285575433533
Distance between circ unitary and qutip trotter 1.788841479862386
Quantum object: dims = [[32], [32]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0


### Fidelity of Circuit vs Exact

In [42]:
circ_with_ini_state = QuantumCircuit(num_qubits, name="H")
circ_with_ini_state.initialize(rand_ini_state, range(num_qubits))  #! Qiskit and Qutip order is not reversed for the state!
initial_state_from_circ = Statevector(circ_with_ini_state).data
energy_initial_state_from_circ_np = initial_state_from_circ.conj().T @ rescaled_hamiltonian_qt.data @ initial_state_from_circ
print('Energy of initial state:')
print(energy_initial_state_from_circ_np.real)

circ_with_ini_state.compose(ham_time_evol, range(num_qubits), inplace=True)
qiskit_end_state = Statevector(circ_with_ini_state).data  # Circ to state

print('Energy after time evolution (should not be changed):')
end_energy = qiskit_end_state.conj().T @ rescaled_hamiltonian_qt.data @ qiskit_end_state
print(end_energy.real)

print('QISKIT CIRCUIT FIDELITY')
trott_evolved_state = Statevector(trott_evolved_state.full())
exact_evolved_state = Statevector(exact_evolved_state.full())

print(f'To qutip exact {state_fidelity(exact_evolved_state, qiskit_end_state)}')
print(f'To qutip Trotter {state_fidelity(trott_evolved_state, qiskit_end_state)}')


Energy of initial state:
0.680166873695157
Energy after time evolution (should not be changed):
0.6800648924522698
QISKIT CIRCUIT FIDELITY
To qutip exact 0.9999785192500749
To qutip Trotter 0.9999999999997207


### Trotterized Hamiltonian comparison, Qiskit vs Exact, Qutip vs Exact

In [35]:
#* LOGM
# logU = logm(trotterized_evolution_unitary)  # log U = i 2pi H T / rescaling_factor  #! It's not unique if any eigenenergy < 0
# trotterized_ham_from_circuit = logU / (1j * T) #- shift * qt.qeye(hamiltonian_qt.shape[0])
# qiskit_trott_ham_spec = np.linalg.eigvalsh(trotterized_ham_from_circuit)
# spectrum_dist = np.linalg.norm(qiskit_trott_ham_spec - exact_spec)
# print(f'Spectrum distance {spectrum_dist}')

#### Qiskit gate angle convention! RXX(-2x) = exp(x)

In [36]:
c = QuantumCircuit(2)
c.rxx(np.pi/2, 0, 1)
op = Operator(c)
print(op.data)
X = qt.sigmax()
XX = qt.tensor(X, X)
op_qt = expm(-1j*np.pi* XX / 4)
print(op_qt)


[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]
